In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
PATH = '../finanz_vs_news/'

In [12]:
df_reu = pd.read_csv(PATH+'reuters_2008.csv.gz', parse_dates=['Time']).set_index('Time')

In [13]:
df_reu.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 771666 entries, 2008-01-02 04:59:53 to 2008-08-20 04:00:00
Data columns (total 1 columns):
Short_Msg    771666 non-null object
dtypes: object(1)
memory usage: 11.8+ MB


In [36]:
#temporary measure to reduce suspicious dates
df_reu = df_reu.loc['2008',:]

In [68]:
import helpers

In [ ]:
helpers.count_sensitive_words()